# Possible Solution: Build A Pipeline

- Combine Your Knowledge of the Website, `requests` and `bs4`
- Automate Your Scraping Process Across Multiple Sites
- Target the Specific Information You Want

## Your tasks:

- Scrape the first 100 available search results
- Generalize your code to allow searching for different locations/jobs
- Pick out the link, title, and location
- Save the results to a file

In [1]:
import requests
from bs4 import BeautifulSoup

---

### Part 1: Inspect

- How do the URLs change when you navigate to the next results page?
- How do the URLs change when you use a different location and/or job title search?
- Which HTML elements contain the link, title, and location of each job?

**Next Page**: The `start=` parameter gets added and incremented by the value of `10` for each additional page. This is because each results page displays 10 job results.

E.g.: <https://www.indeed.com/jobs?q=python&l=new+york&start=20>

**Different Location/Job Title**: The values for the query parameters `q` (for job title) and `l` (for location) change accordingly.

In [18]:
page = requests.get('https://www.indeed.com/jobs?q=python&l=new+york')

**HTML Elements**: A single job posting lives inside of a `div` element with the class name `result`. Inside there are other elements. You can find the specific info you're looking for here:

- **Link**: In the `href` attribute of the `<a>` Element that is a child of the title `<h2>` element
- **Title**: The text of the link in the `<h2>` element which also contains the link URL mentioned above
- **Location**: A `<span>` element with the telling class name `location`

---

### Part 2: Scrape

- Build the code to fetch the first 100 search results. This means you will need to automatically navigate to multiple results pages
- Write functions that allow you specify the job title, location, and amount of results as arguments

In [44]:
page_2 = requests.get('https://www.indeed.com/jobs?q=python&l=new+york&start=20')

Every 10 results means you're on a new page. Let's make that an argument to a function:

In [19]:
def get_jobs(page=1):
    """Fetches the HTML from a search for Python jobs in New York on Indeed.com from a specified page."""
    base_url_indeed = 'https://www.indeed.com/jobs?q=python&l=new+york&start='
    results_start_num = page*10
    url = f'{base_url_indeed}{results_start_num}'
    page = requests.get(url)
    return page

In [20]:
get_jobs(3)

<Response [200]>

In [21]:
get_jobs(4)

<Response [200]>

Great! Let's customize this function some more to allow for different search queries and search locations:

In [23]:
def get_jobs(title, location, page=1):
    """Fetches the HTML from a search for Python jobs in New York on Indeed.com from a specified page."""
    loc = location.replace(' ', '+')  # for multi-part locations
    base_url_indeed = f'https://www.indeed.com/jobs?q={title}&l={loc}&start='
    results_start_num = page*10
    url = f'{base_url_indeed}{results_start_num}'
    page = requests.get(url)
    return page

In [24]:
get_jobs('python', 'new york', 3)

<Response [200]>

With a generalized way of scraping the page done, you can move on to picking out the information you need by parsing the HTML.

---

### Part 3: Parse

- Sieve through your HTML soup to pick out only the job title, link, and location
- Format the results in a readable format (e.g. JSON)
- Save the results to a file

Let's start by getting access to all interesting search results for one page:

In [25]:
site = get_jobs('python', 'new york')

In [26]:
soup = BeautifulSoup(site.content)

In [27]:
results = soup.find(id='resultsCol')

In [28]:
jobs = results.find_all('div', class_='result')

**Job Titles** can be found like this:

In [29]:
job_titles = [job.find('h2').find('a').text.strip() for job in jobs]

In [30]:
job_titles

['Junior Front End / Full Stack Software Engineer',
 'Data Scientist',
 'Data Analysis Intern, Spectrum',
 'Software Engineers & Web Developers',
 'Data Technician (Full- or Part-Time)',
 'Financial Analyst, Revenue Forecasting',
 'Healthcare Data/Reporting Analyst',
 'Online Coding Instructor',
 'Data Journalist',
 'Recruitment Solutions Consultant (REMOTE)']

**Link URLs** need to be assembled, and can be found like this:

In [11]:
base_url = 'https://www.indeed.com'

In [12]:
job_links = [base_url + job.find('h2').find('a')['href'] for job in jobs]

In [13]:
job_links

['https://www.indeed.com/rc/clk?jk=771ec6885c33169f&fccid=00ae4e776f21bc2f&vjs=3',
 'https://www.indeed.com/rc/clk?jk=ab659a231e0d45b4&fccid=b9d4e9eceb3ff4c0&vjs=3',
 'https://www.indeed.com/rc/clk?jk=487b30db63184515&fccid=bf0600f0f252b45b&vjs=3',
 'https://www.indeed.com/rc/clk?jk=f8f068da6dd93fa6&fccid=ca7680692810259a&vjs=3',
 'https://www.indeed.com/rc/clk?jk=da727c0cddda240e&fccid=56a26d4c816e53d1&vjs=3',
 'https://www.indeed.com/rc/clk?jk=75304bc19a74f00b&fccid=00ae4e776f21bc2f&vjs=3',
 'https://www.indeed.com/rc/clk?jk=0094c74f655c2692&fccid=b9d4e9eceb3ff4c0&vjs=3',
 'https://www.indeed.com/rc/clk?jk=b0b5b23a82f838bf&fccid=8844f3447ed4e2d6&vjs=3',
 'https://www.indeed.com/rc/clk?jk=3d07c7b81d80f7cc&fccid=e0ccb12bb145acf5&vjs=3',
 'https://www.indeed.com/rc/clk?jk=1ab7622f6a6d24fd&fccid=10c1f2532daaed15&vjs=3']

**Locations** can be picked out of the soup by their class name:

In [16]:
job_locations = [job.find(class_='location').text for job in jobs]

In [17]:
job_locations

['New York, NY 10005 (Financial District area)',
 'New York State',
 'Florida, NY',
 'New York, NY 10022 (Midtown area)',
 'New York, NY 10003 (Greenwich Village area)',
 'New York, NY 10005 (Financial District area)',
 'New York State',
 'New York, NY 10010 (Flatiron District area)',
 'New York, NY 10013 (SoHo area)',
 'New York, NY 10014 (West Village area)']

Let's assemble all this info into a function, so you can pick out the pieces and save them to a useful data structure:

In [31]:
def parse_info(soup):
    """
    Parses HTML containing job postings and picks out job title, location, and link.
    
    args:
    soup (BeautifulSoup object): A parsed bs4.BeautifulSoup object of a search results page on indeed.com
    
    returns:
    job_list (list): A list of dictionaries containing the title, link, and location of each job posting
    """
    results = soup.find(id='resultsCol')
    jobs = results.find_all('div', class_='result')
    base_url = 'https://www.indeed.com'

    job_list = list()
    for job in jobs:
        title = job.find('h2').find('a').text.strip()
        link = base_url + job.find('h2').find('a')['href']
        location = job.find(class_='location').text
        job_list.append({'title': title, 'link': link, 'location': location})

    return job_list

Let's give it a try:

In [32]:
page = get_jobs('python', 'new_york')

In [33]:
soup = BeautifulSoup(page.content)

In [34]:
results = parse_info(soup)

In [35]:
results

[{'title': 'Junior Front End / Full Stack Software Engineer',
  'link': 'https://www.indeed.com/rc/clk?jk=168a00aa137f6f7f&fccid=6a5673a520e27ea2&vjs=3',
  'location': 'New York, NY'},
 {'title': 'Data Scientist',
  'link': 'https://www.indeed.com/rc/clk?jk=1636faa0308fd08f&fccid=1577085fc2290983&vjs=3',
  'location': 'New York, NY'},
 {'title': 'Data Analysis Intern, Spectrum',
  'link': 'https://www.indeed.com/rc/clk?jk=b0b5b23a82f838bf&fccid=8844f3447ed4e2d6&vjs=3',
  'location': 'New York, NY 10010 (Flatiron District area)'},
 {'title': 'Software Engineers & Web Developers',
  'link': 'https://www.indeed.com/rc/clk?jk=597b9830f39b5420&fccid=bc8bf69154b5fc28&vjs=3',
  'location': 'New York, NY'},
 {'title': 'Data Technician (Full- or Part-Time)',
  'link': 'https://www.indeed.com/rc/clk?jk=da727c0cddda240e&fccid=56a26d4c816e53d1&vjs=3',
  'location': 'New York, NY 10003 (Greenwich Village area)'},
 {'title': 'Financial Analyst, Revenue Forecasting',
  'link': 'https://www.indeed.com

And let's add a final step of generalization:

In [39]:
def get_job_listings(title, location, amount=100):
    results = list()
    for page in range(amount//10):
        site = get_jobs(title, location, page=page)
        soup = BeautifulSoup(site.content)
        page_results = parse_info(soup)
        results += page_results
    return results

In [40]:
r = get_job_listings('python', 'new york', 100)

In [43]:
len(r)

100

In [44]:
r[42]

{'title': 'Senior Systems Engineer',
 'link': 'https://www.indeed.com/rc/clk?jk=8116931b8cfe9ce1&fccid=46a5105b06a6bafb&vjs=3',
 'location': 'New York State'}

### Keep Expanding!

Currently you are only fetching the title, link and location of the job. Change that to get also get the **company name**. Maybe you also want to know the beginning of the **text blurb** what the job is about? You could also build this script out to follow the links you gathered and fetch the individual job listing details pages for even more information.

The sky is the limit, and the more you train, the better you will get at this. :)